In [1]:
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer
import pandas as pd
from nltk import pos_tag, word_tokenize
from textblob import TextBlob, Word
import unicodedata
import regex
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from indicnlp.tokenize import indic_tokenize
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
class Preprocess:
        
    # --------------------------------------- Constructor -------------------------------------------------- 

    def __init__(self,stopword_list):
        self.data_path = ''
        self.stopword_list = stopword_list


    # --------------------------------------- Expand  Concatenations --------------------------------------- 

    def expand_concatenations(self, word):

        if not re.match('[a-zA-Z]+', word) or re.match('/d+',word):
            for i in range(len(word)):
                if not('DEVANAGARI ' in unicodedata.name(word[i])):
                    word = word[:i] if( len(word[i:]) < 2 ) else word[:i] + " " + word[i:]
                    break
        else:
            for i in range(len(word)):
                if ('DEVANAGARI ' in unicodedata.name(word[i])):
                    word = word[i:] if( len(word[:i]) < 2 ) else word[:i] + " " + word[i:]
                    break
        return(word)


    # ----------------------------------------------- Preprocess ------------------------------------------- 

    def clean_text(self,text):
        special_chars = r'''!()-[]{};:'"\,<>./?@#$%^&*_~'''
        stemmer = PorterStemmer()
        lemmatizer = WordNetLemmatizer()
        text = ''.join(x for x in text if x.isprintable())

        text = str(text)

        # Cleaning the urls
        text = re.sub(r'https?://\S+|www\.\S+', '', text)

        # Cleaning the html elements
        text = re.sub(r'<.*?>', '', text)

        # Removing the punctuations
        text = re.sub('[!#?,.:";-@#$%^&*_~<>()-]', ' ', text)

        # Removing stop words
        text = ' '.join([word for word in text.split() if word not in self.stopword_list])

        # Expanding noisy concatenations (Eg: algorithmआणि  -> algorithm आणि ) 
        text = ' '.join([self.expand_concatenations(word) for word in text.split()])

        preprocessed_text = ""

        for word in text.split(): 
            if (re.match('\d+', word)):
                if(word.isnumeric()):
                    preprocessed_text = preprocessed_text + '#N' + " "

            else:
                if(re.match('[a-zA-Z]+', word) and len(word) > 1):
                        word = word.lower()
                        preprocessed_text = preprocessed_text + word + " "

                else:
                    preprocessed_text = preprocessed_text + word + " "

        return preprocessed_text

In [3]:
def analyze_vocab(vocab_words):
    numbers = []
    english_words = []
    marathi_words = []
    for word in vocab_words:
        if re.match('\d+', word):
            numbers.append(word)
        elif re.match('[a-zA-Z]+', word):
            english_words.append(word)
        else:
            marathi_words.append(word)
    return numbers, english_words, marathi_words

def custom_analyzer(text):
    # extract words of at least 1 letters
    words = regex.findall(r'\w{1,}', text)
    for w in words:
        yield w

def bow_vectorize(x_train, x_val):
        bow_vectorizer = CountVectorizer(analyzer=custom_analyzer)
        bow_vectorizer.fit(x_train)
        bow_x_train = bow_vectorizer.transform(x_train)
        bow_x_val = bow_vectorizer.transform(x_val)
        return bow_vectorizer, bow_x_train, bow_x_val

def check_alphanumeric_words(text):
    alpha_numeric_set = set()
    for t in text:
        for word in t.split(): 
            if any(chr.isalpha() for chr in word) and any(chr.isdigit() for chr in word): 
#                 print(word)
                alpha_numeric_set.add(word)
    return alpha_numeric_set

In [4]:
df = pd.read_csv('../dataset/original-dataset/marathi-training-data.csv')
df

,text,label
0,"प्रा . प्रताप हरिदास : होय , मला वाटते की हा ए...",com_tech
1,"तर , विशिष्ट गोष्टींद्वारे , ठराविक कायद्यांद्...",bioche
2,- - - - - - - - - - - - - - - - - - - - - - - ...,cse
3,"तर , आपला अर्धा चिन्ह 9 वाजता असेल .",phy
4,"म्हणून , मी असे म्हणालो की जर शेकडो , हजारो कि...",phy
...,...,...
41992,"जरी आपण डेटा कूटबद्ध केला , तरीही हा मुख्य व्य...",cse
41993,"ते म्हणतात - "" ज्याला पाहण्यासाठी डोळे , ऎकण्य...",com_tech
41994,"प्रथम क्रोनोलॉजिकल , क्रॉनोलॉजी म्हणजे आपल्याल...",com_tech
41995,"त्या थोड्या तपशीलावर येईल , जेणेकरून संपूर्ण ग...",bioche


In [5]:
pp = Preprocess([])
text = df['text'].apply(lambda x : pp.clean_text(x)).tolist()
text

['प्रा प्रताप हरिदास होय मला वाटते की हा एक महत्त्वाचा मुद्दा आहे की भारतीय संदर्भामुळे आपण विशेषतः आमच्या शैक्षणिक प्रक्रियेद्वारे प्रवेश करू शकता जिथे प्रवेश परीक्षा असते आणि जी आपल्याला विभागांमध्ये ठेवते आणि काही आपण त्या विशिष्ट क्रियाकलापांवर अडकले आहात असे कसे वाटते ',
 'तर विशिष्ट गोष्टींद्वारे ठराविक कायद्यांद्वारे विशिष्ट तत्त्वांनी मार्गदर्शित नसल्यास त्या गोष्टी परिभाषित केल्या नसल्यास नियंत्रण करणे खूप कठीण आहे म्हणून त्यात संपूर्ण खाद्यपदार्थ नियंत्रित करण्यासाठी मूळ स्वरुपाचा किंवा प्रत्यक्ष किंवा अप्रत्यक्षपणे हाताळल्यानंतर कोणत्याही स्वरुपाच्या प्रक्रियेनंतर किंवा कोणत्याही स्वरुपाचे नियंत्रण करणे ',
 'म्हणून डेटा कुठे स्थित आहे जेथे विस्तार इत्यादी गोष्टींवर अधिक नियंत्रण नसल्यास प्रवेश विलंब संग्रहित करा या प्रकारच्या परिस्थितीत यासारखे इतर भिन्न समस्या आहेत ',
 'तर आपला अर्धा चिन्ह वाजता असेल ',
 'म्हणून मी असे म्हणालो की जर शेकडो हजारो किंवा लाखो विंडील्स बाहेर टाकत असतील तर त्या ब्लेडच्या पुनर्चक्रणासाठी कोणता मार्ग आहे हे जाणून घेणे महत्वाचे आहे ',
 'मी म्हणालो त

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index
        
print('Tensorflow tokenizer: Unique tokens = {} , Alphanumeric = {}'.format(len(word_index),len(check_alphanumeric_words(word_index))))

num1 , en1 , mar1 = analyze_vocab(word_index)
num1 = set(num1)
en1 = set(en1)
mar1 = set(mar1)
print('#Num = {} , #Eng = {}, #Mar = {} '.format(len(num1),len(en1),len(mar1)))

Tensorflow tokenizer: Unique tokens = 51881 , Alphanumeric = 96
#Num = 0 , #Eng = 936, #Mar = 50945 


In [7]:
indic_nlp_tokens = set()
for record in text:
    tokens = indic_tokenize.trivial_tokenize(record, lang = 'mar')
    for t in tokens:
        indic_nlp_tokens.add(t)


print('IndicNLP tokenizer: Unique tokens = {} , Alphanumeric = {}'.format(len(indic_nlp_tokens),len(check_alphanumeric_words(indic_nlp_tokens))))

num2 , en2 , mar2 = analyze_vocab(indic_nlp_tokens)
num2 = set(num2)
en2 = set(en2)
mar2 = set(mar2)
print('#Num = {} , #Eng = {}, #Mar = {} '.format(len(num2),len(en2),len(mar2)))

IndicNLP tokenizer: Unique tokens = 51908 , Alphanumeric = 96
#Num = 0 , #Eng = 960, #Mar = 50948 


In [14]:
mar_diff = mar1 - mar2
print(len(mar_diff))
mar_diff

2


{'°c', 'σd'}

In [16]:
eng_diff = en2 - en1
print(len(eng_diff))

24


In [10]:
training_data = pd.read_csv("../dataset/original-dataset/marathi-training-data.csv")
training_data['text'] = training_data.text.apply(lambda x: pp.clean_text(x))
x_train = training_data.text.values.tolist()
val_data = pd.read_csv("../dataset/original-dataset/marathi-validation-data.csv")
val_data['text'] = val_data.text.apply(lambda x: pp.clean_text(x))
x_val = val_data.text.values.tolist()

bow_vectorizer, bow_x_train, bow_x_val = bow_vectorize(x_train, x_val)
print(bow_x_train.shape)
print(bow_x_val.shape)

(41997, 51903)
(3780, 51903)


In [11]:
print('CountVectorizer tokenizer: Unique tokens = {} , Alphanumeric = {}'.format(len(bow_vectorizer.get_feature_names()),len(check_alphanumeric_words(bow_vectorizer.get_feature_names()))))

num3 , en3 , mar3 = analyze_vocab(bow_vectorizer.get_feature_names())
num3 = set(num3)
en3 = set(en3)
mar3 = set(mar3)
print('#Num = {} , #Eng = {}, #Mar = {} '.format(len(num3),len(en3),len(mar3)))

CountVectorizer tokenizer: Unique tokens = 51903 , Alphanumeric = 96
#Num = 0 , #Eng = 961, #Mar = 50942 


In [12]:
mar_diff1 = mar1 - mar3
print(len(mar_diff1))
mar_diff1

4


{'°c', 'σd', '।', '–tp'}

In [18]:
eng_diff1 = en3 - en1
print(len(eng_diff1))
eng_diff1

25


{'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'tp'}

In [ ]:
text = "\u200b\u200bअसता I am \u200b\u200b४Atharva \u200b"
text

In [ ]:
text = ''.join(x for x in text if x.isprintable())
text

In [ ]:
if __name__ == '__main__':
   
    df = pd.read_csv('../dataset/original-dataset/marathi-training-data.csv')
    stopword_list = []
    lemmatizer = WordNetLemmatizer()

#     with open ('../Technodifacation/Data/marathi_stopwords.txt','r',encoding='utf') as st:
#         st_content = st.read()
#         st_list = set(st_content.split())
#         stopword_list = st_list
    
    pp = Preprocess([])
    
#     df['text'] = df['text'].apply(lambda x : pp.clean_text(x))
#     sample_text = df.sample()['text'].values[0]

    sample_text = "mस्केलेबल algorithmआणि  एमएनओMOM आयपीIPs  वारेB ०००० किलोमीट विशिष्ट-- 19022323239  ० great 2T2 ,H2O, 9909च, Having Caring Sharing शब्दाचा उच्चार कसा केला गेला आणि 99 Working समन्वय साधण्याचा प्रयत्न करा जेव्हा 87929999 एका बिंदूबरोबर इतर गोष्टींचा एका!!! ११ 00 १ Google computer architecture graphic show.!!!"
    preprocessed_text = pp.clean_text(sample_text)
    print('\nBefore:\t',sample_text,'\n\nAfter:\t',preprocessed_text)